In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 53.5 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt

In [ ]:
def hydrogen_hamiltonian(coordinates):
    """Calculates the qubit Hamiltonian of the hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    return qml.qchem.molecular_hamiltonian(["H", "Li"], coordinates, charge=0, basis="STO-3G",active_electrons=2)[0]


def hf(electrons, num_qubits):
    """Calculates the Hartree-Fock state of the hydrogen molecule.

    Args:
        electrons (int): The number of electrons in the hydrogen molecule.
        num_qubits (int): The number of qubits needed to represent the hydrogen molecule Hamiltonian.

    Returns:
        (numpy.tensor): The HF state.
    """
    # Put your solution here #
    return qml.qchem.hf_state(electrons=electrons, orbitals=num_qubits)

In [ ]:
def num_electrons(charge):
    """The total number of electrons in the hydrogen molecule.

    Args:
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (int): The number of electrons.
    """
    return 2-charge

In [ ]:
def depth(qnode):
    def _fn(*args, **kwargs):
        qnode.construct(args, kwargs)
        return qnode.qtape.get_depth()
    return _fn

In [ ]:
L=1.5
coordinates = np.array([0.0, 0.0, -L/2, 0.0, 0.0, L/2]) #np.array([-L/2/np.sqrt(3), L/2, 0.0, L/np.sqrt(3), 0.0, 0.0,-L/2/np.sqrt(3), -L/2, 0.0])
charge=0
hamiltonian = hydrogen_hamiltonian(np.array(coordinates))
H_matrix=qml.matrix(hamiltonian)

In [ ]:
H_sq=H_matrix.dot(H_matrix)
H_ob_sq=qml.Hermitian(H_sq, wires=range(len(hamiltonian.wires)))

In [ ]:
vals, vecs = np.linalg.eigh(H_matrix)

inds=np.argsort(vals)
eng=vals[inds]
vec=vecs[:,inds]

print('The ground state energy:',eng[0])

The ground state energy: -7.626624465547493


In [ ]:
def vec_check(vec,electron_num):
    for i,e in enumerate(vec):
        if np.abs(e) > 1e-8:
            e_occ=bin(i)[2:]
            lst=' '.join(e_occ).split(' ')
            cv=tuple(map(int, lst))
            e_num=sum(cv)
            if e_num!=electron_num:
                return False
    return True

In [ ]:
sz=qml.qchem.spinz(len(hamiltonian.wires))
SZ=qml.matrix(sz)
Stop=[False,False,False]
elec_num=2
for i in range(1,len(eng)): # Finding first excited state with -1 Sz
    Sz=vec[:,i].dot(SZ.dot(np.transpose(np.conjugate(vec[:,i]))))
    #print(Sz)

    if np.abs(Sz) < 1e-5 and vec_check(vec[:,i],elec_num):
        print('The first excied state energy with sz=0:',eng[i])
        Stop[0]=True

    elif np.abs(Sz-1) < 1e-5 and vec_check(vec[:,i],elec_num):
        print('The first excied state energy with sz=1:',eng[i])
        Stop[1]=True

    elif np.abs(Sz+1) < 1e-5 and vec_check(vec[:,i],elec_num):
        print('The first excied state energy with sz=-1:',eng[i])
        Stop[2]=True

    if np.all(Stop):
        break


The first excied state energy with sz=1: -7.522914627103037
The first excied state energy with sz=0: -7.522914627103034
The first excied state energy with sz=-1: -7.52291462710303


In [ ]:
electrons = 2
num_qubits = len(hamiltonian.wires)

S2 = qml.qchem.spin2(electrons, num_qubits)
sz=qml.qchem.spinz(num_qubits)

singles, doubles = qml.qchem.excitations(electrons, num_qubits)
pool_operators=singles+doubles

num_qubits = len(hamiltonian.wires)
hf_state = hf(electrons, num_qubits)

dev = qml.device("default.qubit", wires=num_qubits)
print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")

bitstr=''.join(map(str, hf_state))

g_trial_stat={bitstr: 1}
e_trial_stat_szp1={'1010000000': 1}
e_trial_stat_szm1={'0101000000': 1}
e_trial_stat_sz0={'1001000000': 1,'0110000000': 1}

def bit_to_vec(bitstring_dict,num_qubits):
    vec=np.zeros(2**num_qubits)
    amp=0
    for i in range(2**num_qubits):
        bitstr=bin(i)[2:]
        if len(bitstr) < num_qubits:
            bitstr=(num_qubits-len(bitstr))*'0'+bitstr

        if bitstr in bitstring_dict:
            vec[i]=bitstring_dict[bitstr]
            amp+=np.abs(bitstring_dict[bitstr])**2
    vec=vec/np.sqrt(amp)
    return vec

g_stat=bit_to_vec(g_trial_stat,num_qubits)

def circuit(state,weights):
    qml.StatePrep(state, wires=range(num_qubits))
    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i], wires=doubles[i])

    return qml.expval(hamiltonian)

def circuit_sz(state,weights):
    qml.StatePrep(state, wires=range(num_qubits))
    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i], wires=doubles[i])

    return qml.expval(sz)


cost_h_fn = qml.QNode(circuit, dev)
Sz_fn = qml.QNode(circuit_sz, dev)

def cost_fn(params):
    return cost_h_fn(g_stat,params)

weights =np.zeros(len(doubles + singles), requires_grad=True)

opt = qml.AdamOptimizer(stepsize=0.5)

i=0
iter=[]
cost_val=[]
Lowest_E=0
best_weights=0

cov=1e-6
E0_pev=0


for _ in range(400):
    weights = opt.step(cost_fn, weights)
    iter.append(i)

    E0=cost_h_fn(g_stat,weights)

    if Lowest_E>E0:
        Lowest_E=E0
        best_weights=weights
    cost_val.append(E0)

    i=i+1
    print('Need iter:',i,' E0:',E0,' Sz:',Sz_fn(g_stat,weights))
    #print('E2:',E2,Sz_fn(e_stat_sz0,weights))
    #print('E3:',E3,Sz_fn(e_stat_szm1,weights))
    if np.abs(E0-E0_pev)<cov:# and np.abs(E1-E1_pev)<cov and np.abs(E2-E2_pev)<cov and np.abs(E3-E3_pev)<cov:
        #print('Need iter:',i,' E0:',E0)
        break

    E0_pev=E0


The original vqe use  224  cnot gates
The original vqe use  624  gates
Need iter: 1  E0: -7.475187554557702  Sz: 0.0
Need iter: 2  E0: -7.53914631149875  Sz: -2.7755575615628914e-17
Need iter: 3  E0: -7.58517297151212  Sz: 2.7755575615628914e-17
Need iter: 4  E0: -7.53149154544308  Sz: 5.551115123125783e-17
Need iter: 5  E0: -7.530700124349723  Sz: 0.0
Need iter: 6  E0: -7.56655319979495  Sz: 2.7755575615628914e-17
Need iter: 7  E0: -7.590039956292802  Sz: 0.0
Need iter: 8  E0: -7.58705149685098  Sz: -8.326672684688674e-17
Need iter: 9  E0: -7.580778730576064  Sz: 0.0
Need iter: 10  E0: -7.585438382869366  Sz: -2.7755575615628914e-17
Need iter: 11  E0: -7.599889175920599  Sz: -2.7755575615628914e-17
Need iter: 12  E0: -7.6126141810066414  Sz: -2.7755575615628914e-17
Need iter: 13  E0: -7.608241253904193  Sz: 2.7755575615628914e-17
Need iter: 14  E0: -7.597549385444611  Sz: 2.7755575615628914e-17
Need iter: 15  E0: -7.5982604705574746  Sz: 0.0
Need iter: 16  E0: -7.609303957356188  Sz: 

In [ ]:
def circuit_h_sq(state,weights):
    qml.StatePrep(state, wires=range(num_qubits))
    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i], wires=doubles[i])

    return qml.expval(H_ob_sq)

e_stat_sz0=bit_to_vec(e_trial_stat_sz0,num_qubits)

cost_hsq_fn = qml.QNode(circuit_h_sq, dev)
om=-7.545
def cost_fn(params):
    return cost_hsq_fn(e_stat_sz0,params)-cost_h_fn(e_stat_sz0,params)**2#cost_hsq_fn(e_stat_sz0,params)-2*om*cost_h_fn(e_stat_sz0,params)+(om)**2

weights =np.zeros(len(doubles + singles), requires_grad=True)

opt = qml.AdamOptimizer(stepsize=0.5)

i=0
iter=[]
cost_val=[]
Lowest_E=0
best_weights=0

cov=1e-6
E0_pev=0


for _ in range(400):
    weights = opt.step(cost_fn, weights)
    iter.append(i)

    E0=cost_h_fn(e_stat_sz0,weights)

    if Lowest_E>E0:
        Lowest_E=E0
        best_weights=weights
    cost_val.append(E0)

    i=i+1
    print('Need iter:',i,' E0:',E0,' Sz:',Sz_fn(e_stat_sz0,weights))
    if np.abs(E0-E0_pev)<cov:
        break
    E0_pev=E0

Need iter: 1  E0: -7.517692413936705  Sz: 0.0
Need iter: 2  E0: -7.473994891945469  Sz: 2.7755575615628914e-17
Need iter: 3  E0: -7.508167799333992  Sz: 2.7755575615628914e-17
Need iter: 4  E0: -7.505571470283631  Sz: -2.7755575615628914e-17
Need iter: 5  E0: -7.514240647016118  Sz: -2.7755575615628914e-17
Need iter: 6  E0: -7.514859878802045  Sz: 2.7755575615628914e-17
Need iter: 7  E0: -7.514057628500327  Sz: 2.7755575615628914e-17
Need iter: 8  E0: -7.512876380086055  Sz: 2.7755575615628914e-17
Need iter: 9  E0: -7.514807382059969  Sz: -2.7755575615628914e-17
Need iter: 10  E0: -7.517854947351228  Sz: 5.551115123125783e-17
Need iter: 11  E0: -7.517022463395421  Sz: 2.7755575615628914e-17
Need iter: 12  E0: -7.517291887442627  Sz: -5.551115123125783e-17
Need iter: 13  E0: -7.514389158083335  Sz: 2.7755575615628914e-17
Need iter: 14  E0: -7.516614127682341  Sz: 2.7755575615628914e-17
Need iter: 15  E0: -7.516838800624543  Sz: -2.7755575615628914e-17
Need iter: 16  E0: -7.5192953206584

In [51]:
electrons = 2
num_qubits = len(hamiltonian.wires)

singles, doubles = qml.qchem.excitations(electrons, num_qubits)
pool_operators=singles+doubles

num_qubits = len(hamiltonian.wires)
hf_state = hf(electrons, num_qubits)

def circuit_1(state,weights,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(weights[i], wires=excitation)
        else:
            qml.SingleExcitation(weights[i], wires=excitation)
    return qml.expval(hamiltonian)

def circuit_sz(state,weights,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(weights[i], wires=excitation)
        else:
            qml.SingleExcitation(weights[i], wires=excitation)
    return qml.expval(sz)

def circuit_h_sq(state,weights,excitations):

    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(weights[i], wires=excitation)
        else:
            qml.SingleExcitation(weights[i], wires=excitation)

    return qml.expval(H_ob_sq)

dev = qml.device("default.qubit", wires=num_qubits)#

cost_h_fn = qml.QNode(circuit_1, dev, expansion_strategy="device")
cost_hsq_fn = qml.QNode(circuit_h_sq, dev, expansion_strategy="device")
Sz_fn = qml.QNode(circuit_sz, dev)

def cost_fn(params, excitations):
    return cost_hsq_fn(e_stat_sz0,params, excitations)-cost_h_fn(e_stat_sz0,params, excitations)**2


epochs=50

energy=[]
operator_circuits=[]# doubles_select+singles_select

weights = [] #[0.0] * len(operator_circuits)
weights=np.array(weights)

opt = opt = qml.GradientDescentOptimizer(stepsize=0.25)

circuit_gradient = qml.grad(cost_fn, argnum=0)


E0 = cost_h_fn(e_stat_sz0,weights, operator_circuits)

print(f"Epoch = 0, G Energy = {E0:.8f} Ha")

print("Number of gates = {}\n".format(len(operator_circuits)))

configs=qml.specs(cost_h_fn)(e_stat_sz0,weights, excitations=operator_circuits)

#print(configs)
print("Circuit depth {}\n".format(configs['resources'].depth))

conv=1e-7
E0_pev=0
E1_pev=0
E2_pev=0
E3_pev=0
for n in range(epochs):

    Test_set=operator_circuits+pool_operators
    Test_params=list(weights)+[0.0]*len(pool_operators)
    Test_params=np.array(Test_params)

    test_grads = circuit_gradient(Test_params,Test_set)
    test_grads = test_grads[len(weights):]
    maxpos = np.argmax(np.abs(test_grads))

    max_op=pool_operators[maxpos]
    operator_circuits.append(max_op)
    weights=np.append(weights, 0.0)

    weights,_ = opt.step(cost_fn, weights, operator_circuits) # Step 11.
    E0 = cost_h_fn(e_stat_sz0,weights,operator_circuits)


    print(f"Epoch = {n+1}, G Energy = {E0:.8f} Ha, Sz = {Sz_fn(e_stat_sz0,weights,operator_circuits):.8f} Ha,")
    print("Number of gates = {}\n".format(len(operator_circuits)))
    configs=qml.specs(cost_h_fn)(e_stat_sz0,weights, excitations=operator_circuits)
    print("Circuit depth {}\n".format(configs['resources'].depth))

    if np.abs(E0-E0_pev)<cov:
        break

    E0_pev=E0

Epoch = 0, G Energy = -7.49074895 Ha
Number of gates = 0

Circuit depth 1

Epoch = 1, G Energy = -7.49142202 Ha, Sz = 0.00000000 Ha,
Number of gates = 1

Circuit depth 2

Epoch = 2, G Energy = -7.49274112 Ha, Sz = -0.00000000 Ha,
Number of gates = 2

Circuit depth 2

Epoch = 3, G Energy = -7.49463861 Ha, Sz = 0.00000000 Ha,
Number of gates = 3

Circuit depth 3

Epoch = 4, G Energy = -7.49697734 Ha, Sz = 0.00000000 Ha,
Number of gates = 4

Circuit depth 3

Epoch = 5, G Energy = -7.49963005 Ha, Sz = -0.00000000 Ha,
Number of gates = 5

Circuit depth 4

Epoch = 6, G Energy = -7.50244999 Ha, Sz = 0.00000000 Ha,
Number of gates = 6

Circuit depth 4

Epoch = 7, G Energy = -7.50528687 Ha, Sz = 0.00000000 Ha,
Number of gates = 7

Circuit depth 5

Epoch = 8, G Energy = -7.50798171 Ha, Sz = -0.00000000 Ha,
Number of gates = 8

Circuit depth 5

Epoch = 9, G Energy = -7.51042801 Ha, Sz = 0.00000000 Ha,
Number of gates = 9

Circuit depth 6

Epoch = 10, G Energy = -7.51254961 Ha, Sz = 0.00000000 Ha,